In [4]:
!pip install pandas
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: C:\Users\elson\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: C:\Users\elson\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import joblib

In [6]:
df_weather = pd.read_csv("../GeneralData/outfit_combinations_weather_shuffled.csv")
df_style = pd.read_csv("../GeneralData/balanced_fashion_dataset.csv")

In [17]:
print(df_style.head())

       top    bottom    outer styleTop styleBottom  styleOuter styleUser  \
0  t-shirt    shorts   hoodie   formal     vintage      casual    formal   
1  t-shirt  trousers  sweater   sporty  streetwear  streetwear   vintage   
2  t-shirt    jogger  sweater  vintage      sporty      sporty    formal   
3  t-shirt    jogger   hoodie   formal      formal      sporty    formal   
4    shirt     jeans  sweater   casual      casual     vintage    casual   

   userLike  
0         0  
1         0  
2         0  
3         1  
4         1  


In [7]:
# Prepare Model 1 (Weather suitability)
X1 = df_weather.drop("recommended", axis=1)
y1 = df_weather["recommended"]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

label_encoders_1 = {}
for col in X1_train.columns:
    le = LabelEncoder()
    X1_train[col] = le.fit_transform(X1_train[col])
    label_encoders_1[col] = le

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X1_train, y1_train)


RandomForestClassifier(random_state=42)

In [9]:
# Prepare Model 2 (Style suitability)
X2 = df_style.drop(columns=['userLike'])
y2 = df_style['userLike']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

label_encoders_2 = {}
for col in X2_train.columns:
    le = LabelEncoder()
    X2_train[col] = le.fit_transform(X2_train[col])
    label_encoders_2[col] = le

param_grid = {'max_depth': [5, 10, 15, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 5]}
dt_model = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X2_train, y2_train)

best_dt_model = grid_search.best_estimator_

In [16]:
print(X2_train)


      top  bottom  outer  styleTop  styleBottom  styleOuter  styleUser
2905    1       1      2         3            1           1          1
530     1       0      0         4            3           4          0
3290    1       0      2         1            4           4          4
996     1       3      3         4            2           2          2
3905    2       3      2         4            4           4          4
...   ...     ...    ...       ...          ...         ...        ...
5734    2       1      3         1            2           0          0
5191    0       4      3         3            3           2          3
5390    2       0      3         0            1           0          4
860     1       7      4         0            4           0          0
7270    2       5      3         3            0           3          3

[6825 rows x 7 columns]


In [12]:
joblib.dump(rf_model,"GeneralModelWeather.pkl")
joblib.dump(best_dt_model,"GeneralModelStyle.pkl")
joblib.dump(label_encoders_1,"LabelEncoderWeather.pkl")
joblib.dump(label_encoders_2,"LabelEncoderStyle.pkl")

['LabelEncoderStyle.pkl']

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Load datasets
df_weather = pd.read_csv("outfit_combinations_weather_shuffled.csv")
df_style = pd.read_csv("balanced_fashion_dataset.csv")

# Prepare Model 1 (Weather suitability)
X1 = df_weather.drop("recommended", axis=1)
y1 = df_weather["recommended"]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

label_encoders_1 = {}
for col in X1_train.columns:
    le = LabelEncoder()
    X1_train[col] = le.fit_transform(X1_train[col])
    X1_test[col] = le.transform(X1_test[col])
    label_encoders_1[col] = le

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X1_train, y1_train)
X1_test['weather_score'] = rf_model.predict_proba(X1_test)[:, 1]

# Prepare Model 2 (Style suitability)
X2 = df_style.drop(columns=['userLike'])
y2 = df_style['userLike']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

label_encoders_2 = {}
for col in X2_train.columns:
    le = LabelEncoder()
    X2_train[col] = le.fit_transform(X2_train[col])
    X2_test[col] = le.transform(X2_test[col])
    label_encoders_2[col] = le

param_grid = {'max_depth': [5, 10, 15, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 5]}
dt_model = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(dt_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X2_train, y2_train)

best_dt_model = grid_search.best_estimator_
X2_test['style_score'] = best_dt_model.predict_proba(X2_test)[:, 1]

# Merge based on outfit features (top, bottom, outer)
merged = pd.merge(X1_test, X2_test, on=['top', 'bottom', 'outer'], how='inner')

# Compute final weighted score
merged['final_score'] = 0.6 * merged['weather_score'] + 0.4 * merged['style_score']

# Decode categorical values before displaying recommendations
for col in ['top', 'bottom', 'outer', 'weather']:
    if col in label_encoders_1:
        merged[col] = merged[col].apply(lambda x: label_encoders_1[col].inverse_transform([x])[0])

for col in ['styleTop', 'styleBottom', 'styleOuter', 'styleUser']:
    if col in label_encoders_2:
        merged[col] = merged[col].apply(lambda x: label_encoders_2[col].inverse_transform([x])[0])

# Output ranked recommendations with decoded values
recommendations = merged[['top', 'styleTop', 'bottom', 'styleBottom', 'outer', 'styleOuter', 
                          'weather', 'weather_score', 'styleUser', 'style_score', 'final_score']]
recommendations = recommendations.sort_values(by="final_score", ascending=False)
recommendations.head(1000)  # Tampilkan 1000 rekomendasi terbaik

# # Output ranked recommendations
# recommendations = merged[['top', 'styleTop', 'bottom', 'styleBottom', 'outer', 'styleOuter', 'weather', 'weather_score', 'styleUser', 'style_score', 'final_score']]
# recommendations = recommendations.sort_values(by="final_score", ascending=False)
# recommendations.head(1000) # Top 10 outfit recommendations